GBIF.org (26 October 2024) GBIF Occurrence Download  https://doi.org/10.15468/dl.gu5t27

## Questions 1 + 2

In [2]:
import pandas as pd

In [21]:
data = pd.read_csv('kenya_birds.csv', sep='\t')
data.head()

/var/folders/cc/h352fz_d24n9tjfbnmrxstbh0000gn/T/ipykernel_8789/2491437449.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('kenya_birds.csv', sep='\t')


,gbifID,datasetKey,occurrenceID,kingdom,phylum,class,order,family,genus,species,...,identifiedBy,dateIdentified,license,rightsHolder,recordedBy,typeStatus,establishmentMeans,lastInterpreted,mediaType,issue
0,1986821111,d1e8a7af-097a-4ca8-aa91-62d3ff834711,d33e1483-ad6a-4aa2-95bd-5d4568a3cd3b,Animalia,Chordata,Aves,Passeriformes,Zosteropidae,Zosterops,Zosterops silvanus,...,NaN,NaN,CC_BY_4_0,NaN,NaN,NaN,NaN,2024-03-21T12:54:52.894Z,NaN,COORDINATE_ROUNDED;GEODETIC_DATUM_ASSUMED_WGS8...
1,1986821110,d1e8a7af-097a-4ca8-aa91-62d3ff834711,d12a2b54-f12a-40a7-bfb5-90b4998af7d3,Animalia,Chordata,Aves,Passeriformes,Zosteropidae,Zosterops,Zosterops senegalensis,...,NaN,NaN,CC_BY_4_0,NaN,NaN,NaN,NaN,2024-03-21T12:54:54.972Z,NaN,COORDINATE_ROUNDED;GEODETIC_DATUM_ASSUMED_WGS8...
2,1986821109,d1e8a7af-097a-4ca8-aa91-62d3ff834711,ff92601a-f8b9-4db0-90a1-e4d0b233ace0,Animalia,Chordata,Aves,Passeriformes,Zosteropidae,Zosterops,Zosterops senegalensis,...,NaN,NaN,CC_BY_4_0,NaN,NaN,NaN,NaN,2024-03-21T12:54:57.153Z,NaN,COORDINATE_ROUNDED;GEODETIC_DATUM_ASSUMED_WGS8...
3,1986821108,d1e8a7af-097a-4ca8-aa91-62d3ff834711,a6bf4a1c-5c59-4a23-a18a-4ca9e6c36342,Animalia,Chordata,Aves,Passeriformes,Zosteropidae,Zosterops,Zosterops silvanus,...,NaN,NaN,CC_BY_4_0,NaN,NaN,NaN,NaN,2024-03-21T12:55:11.136Z,NaN,COORDINATE_ROUNDED;GEODETIC_DATUM_ASSUMED_WGS8...
4,1986821107,d1e8a7af-097a-4ca8-aa91-62d3ff834711,215c7fe0-9e39-4497-86b5-87532c478ef8,Animalia,Chordata,Aves,Passeriformes,Zosteropidae,Zosterops,Zosterops silvanus,...,NaN,NaN,CC_BY_4_0,NaN,NaN,NaN,NaN,2024-03-21T12:54:51.116Z,NaN,COORDINATE_ROUNDED;GEODETIC_DATUM_ASSUMED_WGS8...


First, let's remove rows that are irrelevant for our analysis:

In [22]:
df1 = data.drop(['gbifID', 'datasetKey', 'occurrenceID', 'verbatimScientificNameAuthorship', 'countryCode', 'publishingOrgKey', 'institutionCode', 'collectionCode', 
                 'catalogNumber', 'recordNumber', 'license', 'rightsHolder', 'recordedBy', 'typeStatus', 'establishmentMeans', 'lastInterpreted'], axis=1)

df1.head()

,kingdom,phylum,class,order,family,genus,species,infraspecificEpithet,taxonRank,scientificName,...,day,month,year,taxonKey,speciesKey,basisOfRecord,identifiedBy,dateIdentified,mediaType,issue
0,Animalia,Chordata,Aves,Passeriformes,Zosteropidae,Zosterops,Zosterops silvanus,NaN,SPECIES,"Zosterops silvanus J.L.Peters & Loveridge, 1935",...,9,11,2017,6100772,6100772.0,HUMAN_OBSERVATION,NaN,NaN,NaN,COORDINATE_ROUNDED;GEODETIC_DATUM_ASSUMED_WGS8...
1,Animalia,Chordata,Aves,Passeriformes,Zosteropidae,Zosterops,Zosterops senegalensis,NaN,SPECIES,"Zosterops senegalensis Bonaparte, 1850",...,1,6,2017,2489344,2489344.0,HUMAN_OBSERVATION,NaN,NaN,NaN,COORDINATE_ROUNDED;GEODETIC_DATUM_ASSUMED_WGS8...
2,Animalia,Chordata,Aves,Passeriformes,Zosteropidae,Zosterops,Zosterops senegalensis,NaN,SPECIES,"Zosterops senegalensis Bonaparte, 1850",...,1,6,2017,2489344,2489344.0,HUMAN_OBSERVATION,NaN,NaN,NaN,COORDINATE_ROUNDED;GEODETIC_DATUM_ASSUMED_WGS8...
3,Animalia,Chordata,Aves,Passeriformes,Zosteropidae,Zosterops,Zosterops silvanus,NaN,SPECIES,"Zosterops silvanus J.L.Peters & Loveridge, 1935",...,6,10,2017,6100772,6100772.0,HUMAN_OBSERVATION,NaN,NaN,NaN,COORDINATE_ROUNDED;GEODETIC_DATUM_ASSUMED_WGS8...
4,Animalia,Chordata,Aves,Passeriformes,Zosteropidae,Zosterops,Zosterops silvanus,NaN,SPECIES,"Zosterops silvanus J.L.Peters & Loveridge, 1935",...,6,10,2017,6100772,6100772.0,HUMAN_OBSERVATION,NaN,NaN,NaN,COORDINATE_ROUNDED;GEODETIC_DATUM_ASSUMED_WGS8...


Now, let's remove columns that are a form of duplicate of other columns:

In [25]:
df2 = df1.drop(['day', 'month', 'year', 'dateIdentified' # the date is contained in 'eventDate'
                ], axis=1)

df2.head()

,kingdom,phylum,class,order,family,genus,species,infraspecificEpithet,taxonRank,scientificName,...,elevationAccuracy,depth,depthAccuracy,eventDate,taxonKey,speciesKey,basisOfRecord,identifiedBy,mediaType,issue
0,Animalia,Chordata,Aves,Passeriformes,Zosteropidae,Zosterops,Zosterops silvanus,NaN,SPECIES,"Zosterops silvanus J.L.Peters & Loveridge, 1935",...,NaN,NaN,NaN,2017-11-09,6100772,6100772.0,HUMAN_OBSERVATION,NaN,NaN,COORDINATE_ROUNDED;GEODETIC_DATUM_ASSUMED_WGS8...
1,Animalia,Chordata,Aves,Passeriformes,Zosteropidae,Zosterops,Zosterops senegalensis,NaN,SPECIES,"Zosterops senegalensis Bonaparte, 1850",...,NaN,NaN,NaN,2017-06-01,2489344,2489344.0,HUMAN_OBSERVATION,NaN,NaN,COORDINATE_ROUNDED;GEODETIC_DATUM_ASSUMED_WGS8...
2,Animalia,Chordata,Aves,Passeriformes,Zosteropidae,Zosterops,Zosterops senegalensis,NaN,SPECIES,"Zosterops senegalensis Bonaparte, 1850",...,NaN,NaN,NaN,2017-06-01,2489344,2489344.0,HUMAN_OBSERVATION,NaN,NaN,COORDINATE_ROUNDED;GEODETIC_DATUM_ASSUMED_WGS8...
3,Animalia,Chordata,Aves,Passeriformes,Zosteropidae,Zosterops,Zosterops silvanus,NaN,SPECIES,"Zosterops silvanus J.L.Peters & Loveridge, 1935",...,NaN,NaN,NaN,2017-10-06,6100772,6100772.0,HUMAN_OBSERVATION,NaN,NaN,COORDINATE_ROUNDED;GEODETIC_DATUM_ASSUMED_WGS8...
4,Animalia,Chordata,Aves,Passeriformes,Zosteropidae,Zosterops,Zosterops silvanus,NaN,SPECIES,"Zosterops silvanus J.L.Peters & Loveridge, 1935",...,NaN,NaN,NaN,2017-10-06,6100772,6100772.0,HUMAN_OBSERVATION,NaN,NaN,COORDINATE_ROUNDED;GEODETIC_DATUM_ASSUMED_WGS8...


Let's check for missing values:

In [26]:
missing_counts = df2.isnull().sum()

total_rows = len(df2)
missing_percentage = (missing_counts / total_rows) * 100

missing_summary = pd.DataFrame({
    'Column Name': missing_counts.index,
    'Missing Count': missing_counts.values,
    'Missing Percentage': missing_percentage.values
})

print(missing_summary)

                      Column Name  Missing Count  Missing Percentage
0                         kingdom              0            0.000000
1                          phylum              0            0.000000
2                           class              0            0.000000
3                           order              0            0.000000
4                          family              0            0.000000
5                           genus              0            0.000000
6                         species            185            0.206400
7            infraspecificEpithet          89593           99.956489
8                       taxonRank              0            0.000000
9                  scientificName              0            0.000000
10         verbatimScientificName              0            0.000000
11                       locality          89632          100.000000
12                  stateProvince          89632          100.000000
13               occurrenceStatus 

Let's remove the columns with nearly all, or all values missing:

In [27]:
df3 = df2.drop(['infraspecificEpithet', 'locality', 'stateProvince', 'coordinateUncertaintyInMeters', 'coordinatePrecision', 'elevation', 'elevationAccuracy',
                'depth', 'depthAccuracy', 'identifiedBy', 'mediaType'], axis=1)

df3.head()

,kingdom,phylum,class,order,family,genus,species,taxonRank,scientificName,verbatimScientificName,occurrenceStatus,individualCount,decimalLatitude,decimalLongitude,eventDate,taxonKey,speciesKey,basisOfRecord,issue
0,Animalia,Chordata,Aves,Passeriformes,Zosteropidae,Zosterops,Zosterops silvanus,SPECIES,"Zosterops silvanus J.L.Peters & Loveridge, 1935",Zosterops silvanus,PRESENT,1.0,-3.432285,38.341012,2017-11-09,6100772,6100772.0,HUMAN_OBSERVATION,COORDINATE_ROUNDED;GEODETIC_DATUM_ASSUMED_WGS8...
1,Animalia,Chordata,Aves,Passeriformes,Zosteropidae,Zosterops,Zosterops senegalensis,SPECIES,"Zosterops senegalensis Bonaparte, 1850",Zosterops senegalensis,PRESENT,1.0,0.206000,34.884667,2017-06-01,2489344,2489344.0,HUMAN_OBSERVATION,COORDINATE_ROUNDED;GEODETIC_DATUM_ASSUMED_WGS8...
2,Animalia,Chordata,Aves,Passeriformes,Zosteropidae,Zosterops,Zosterops senegalensis,SPECIES,"Zosterops senegalensis Bonaparte, 1850",Zosterops senegalensis,PRESENT,4.0,-0.558500,36.523667,2017-06-01,2489344,2489344.0,HUMAN_OBSERVATION,COORDINATE_ROUNDED;GEODETIC_DATUM_ASSUMED_WGS8...
3,Animalia,Chordata,Aves,Passeriformes,Zosteropidae,Zosterops,Zosterops silvanus,SPECIES,"Zosterops silvanus J.L.Peters & Loveridge, 1935",Zosterops silvanus,PRESENT,1.0,-3.399601,38.329516,2017-10-06,6100772,6100772.0,HUMAN_OBSERVATION,COORDINATE_ROUNDED;GEODETIC_DATUM_ASSUMED_WGS8...
4,Animalia,Chordata,Aves,Passeriformes,Zosteropidae,Zosterops,Zosterops silvanus,SPECIES,"Zosterops silvanus J.L.Peters & Loveridge, 1935",Zosterops silvanus,PRESENT,1.0,-3.380438,38.353286,2017-10-06,6100772,6100772.0,HUMAN_OBSERVATION,COORDINATE_ROUNDED;GEODETIC_DATUM_ASSUMED_WGS8...
